In [1]:
import pandas as pd
import numpy as np

In [2]:
dev_nominal = pd.read_csv("dev_data_binarylist.csv", encoding = 'utf-8')

In [4]:
dev_nominal.shape

(1836, 3118)

In [33]:
columns = dev_nominal.columns
columns_other = dev_nominal.columns[0:18]
columns = [x for x in columns if "-" in x]
aggregate_events = [1,11,21,31,41,51,61,71,81,91]
aggregate_dic = {}
for a in aggregate_events:
    raw_list = []
    for i in range(a,a+10):
        col = [x.split("-")[1] for x in columns if str(i)+"-" in x]
        raw_list.extend(col)
    raw_list= list(set(raw_list))
    aggregate_dic[a] = raw_list

In [40]:
def process_row (row,aggregate_dic):
    row_dict = {}
    for k in aggregate_dic.keys():
        columns = aggregate_dic[k]
        for i in range(k,k+10):
            for c in columns:
                c_name = str(k)+"_"+str(k+10)+"_"+c.replace("_binary","")
                if row[str(i)+"-"+c] > 0: 
                    if c_name in row_dict.keys():
                        row_dict[c_name] = row_dict[c_name] + int(row[str(i)+"-"+c])
                    else:
                        row_dict[c_name] =int(row[str(i)+"-"+c])
    return row_dict

In [24]:
row_dict = process_row(dev_nominal.iloc[[0]],aggregate_dic)
row_dict

{'1_11_home_event': 5,
 '1_11_Substitution_EventType': 0,
 '1_11_None_AssistMethod': 6,
 '1_11_YellowCard_EventType': 0,
 '1_11_Blocked_ShotOutcome': 0,
 '1_11_HeadedPass_AssistMethod': 0,
 '1_11_time': 59,
 '1_11_fast_break': 0,
 '1_11_HitThePost_ShotOutcome': 0,
 '1_11_HandBall_EventType': 0,
 '1_11_Offside_EventType': 1,
 '1_11_Attempt(shot)_EventType': 5,
 '1_11_Corner_Situation': 0,
 '1_11_OwnGoal_EventType2': 0,
 '1_11_OnTarget_ShotOutcome': 2,
 '1_11_PenaltyConceded_EventType': 0,
 '1_11_KeyPass_EventType2': 4,
 '1_11_FreeKickWon_EventType': 2,
 '1_11_SetPiece_Situation': 0,
 '1_11_OpenPlay_Situation': 4,
 '1_11_SecondYellowCard_EventType': 0,
 '1_11_OffTarget_ShotOutcome': 3,
 '1_11_SendingOff_EventType2': 0,
 '1_11_ThroughBall_AssistMethod': 0,
 '1_11_Corner_EventType': 0,
 '1_11_Cross_AssistMethod': 0,
 '1_11_Pass_AssistMethod': 4,
 '1_11_FreeKick_Situation': 1,
 '1_11_StraightRedCard_EventType': 0,
 '1_11_FailedThroughBall_EventType2': 1,
 '1_11_Foul_EventType': 2,
 '11_21_h

In [16]:
dev_nominal.iloc[[0]]

,id_odsp,season,fthg,ftag,odd_h,odd_d,odd_a,class,D1_league_binary,E0_league_binary,...,100-OnTarget_ShotOutcome_binary,100-Cross_AssistMethod_binary,100-HeadedPass_AssistMethod_binary,100-None_AssistMethod_binary,100-Pass_AssistMethod_binary,100-ThroughBall_AssistMethod_binary,100-Corner_Situation_binary,100-FreeKick_Situation_binary,100-OpenPlay_Situation_binary,100-SetPiece_Situation_binary
0,GIqnQgzL/,2016,1,1,1.58,4.6,7.7,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
df_new = pd.DataFrame()
for index, row in dev_nominal.iterrows():
    row_dict = process_row(row,aggregate_dic)
    new_row = pd.DataFrame(row_dict, index=[index,])
    for c in columns_other:
        new_row[c] = row[c]
    df_new =  pd.concat([df_new, new_row])

In [48]:
df_new= df_new.sort_index(axis = 1)

In [49]:
df_new.to_csv("dev_data_aggregate.csv", encoding = 'utf-8',index=False)